# Advanced Self-Evaluating Agentic AI System Tutorial

This notebook demonstrates how to build and use an advanced agentic AI system with self-evaluation capabilities, based on the [MarkTechPost tutorial](https://www.marktechpost.com/2026/01/17/how-to-build-a-self-evaluating-agentic-ai-system-with-llamaindex-and-openai-using-retrieval-tool-use-and-automated-quality-checks/).

## Features

- **Self-Evaluation**: Automatic quality assessment with faithfulness and relevancy metrics
- **Advanced RAG**: Retrieval-augmented generation with evidence-based responses
- **Knowledge Management**: Dynamic knowledge base with document indexing
- **Tool Integration**: Structured tool use with validation
- **Performance Monitoring**: Comprehensive logging and statistics

## Setup and Installation

In [ ]:
# Install required packages
!pip install -q -U llama-index llama-index-llms-openai llama-index-embeddings-openai nest_asyncio
!pip install -q python-dotenv click rich pydantic pyyaml structlog tenacity numpy pandas

In [ ]:
import os
import asyncio
import nest_asyncio
from getpass import getpass
import sys

# Enable nested async loops
nest_asyncio.apply()

# Add the src directory to Python path
sys.path.append('../src')

# Set up OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

## Basic Usage: Quick Start

In [ ]:
from agentic_ai import SystemConfig, AdvancedAgenticAI

# Create configuration
config = SystemConfig()
config.openai.api_key = os.environ["OPENAI_API_KEY"]
config.openai.model = "gpt-4o-mini"
config.openai.temperature = 0.2

# Initialize the AI system
ai_system = AdvancedAgenticAI(config)
session_id = ai_system.start_session("tutorial_session")

print(f"🚀 AI system initialized with session: {session_id}")

## Simple Query Processing

In [ ]:
# Process a simple query
query = "What are the key components of a reliable RAG system?"

response = await ai_system.process_query(query)

print(f"Query: {query}")
print(f"\nResponse: {response.response}")

if response.evaluation:
    print(f"\n📊 Evaluation:")
    print(f"  Overall Score: {response.evaluation.overall_score:.2f}")
    print(f"  Faithfulness: {response.evaluation.faithfulness_score:.2f}")
    print(f"  Relevancy: {response.evaluation.relevancy_score:.2f}")
    print(f"  Status: {'✅ PASSED' if response.evaluation.passed_thresholds else '❌ NEEDS IMPROVEMENT'}")

## Knowledge Base Management

In [ ]:
# Add custom knowledge to the system
custom_knowledge = [
    {
        "text": "Advanced evaluation metrics for AI systems include perplexity, BLEU scores, "
               "ROUGE scores, and human evaluation. For agentic systems, additional metrics "
               "include tool usage accuracy, reasoning coherence, and factual consistency.",
        "title": "AI Evaluation Metrics",
        "tags": ["evaluation", "metrics", "assessment"]
    },
    {
        "text": "Prompt engineering for agentic AI involves designing structured prompts that "
               "guide the agent's reasoning process, tool selection, and response generation. "
               "Best practices include clear role definition, step-by-step instructions, and "
               "examples of desired behavior.",
        "title": "Agentic Prompt Engineering",
        "tags": ["prompts", "engineering", "agents"]
    }
]

for knowledge in custom_knowledge:
    ai_system.add_knowledge_from_text(
        knowledge["text"],
        knowledge["title"],
        knowledge["tags"]
    )

# Check knowledge base statistics
kb_stats = ai_system.knowledge_base.get_stats()
print(f"📚 Knowledge Base Stats:")
print(f"  Total Documents: {kb_stats.total_documents}")
print(f"  Total Nodes: {kb_stats.total_nodes}")
print(f"  Document Types: {kb_stats.document_types}")

## Knowledge Base Search

In [ ]:
# Search the knowledge base
search_query = "evaluation metrics for AI"
results = ai_system.knowledge_base.search(search_query, top_k=3)

print(f"🔍 Search results for: '{search_query}'\n")

for i, result in enumerate(results, 1):
    score = result.get('score', 0.0)
    content = result['content'][:200] + "..." if len(result['content']) > 200 else result['content']
    metadata = result.get('metadata', {})
    title = metadata.get('title', 'Unknown')
    
    print(f"{i}. Score: {score:.3f} | Source: {title}")
    print(f"   {content}\n")

## Advanced Query Processing with Self-Evaluation

In [ ]:
# Process a complex query that requires reasoning
complex_query = (
    "Design a comprehensive evaluation framework for agentic AI systems that includes "
    "both automated metrics and human feedback. Explain how each component contributes "
    "to overall system reliability."
)

print(f"Complex Query: {complex_query}\n")

response = await ai_system.process_query(complex_query)

print("="*60)
print("AGENT RESPONSE:")
print("="*60)
print(response.response)

if response.evaluation:
    print("\n" + "="*60)
    print("EVALUATION RESULTS:")
    print("="*60)
    
    eval_result = response.evaluation
    
    print(f"Overall Score: {eval_result.overall_score:.2f}")
    print(f"Faithfulness: {eval_result.faithfulness_score:.2f}")
    print(f"Relevancy: {eval_result.relevancy_score:.2f}")
    print(f"Answer Quality: {eval_result.answer_relevancy_score:.2f}")
    print(f"Context Quality: {eval_result.context_relevancy_score:.2f}")
    print(f"Status: {'✅ PASSED' if eval_result.passed_thresholds else '❌ NEEDS IMPROVEMENT'}")
    
    if eval_result.detailed_feedback.get('recommendations'):
        print("\n💡 Recommendations:")
        for rec in eval_result.detailed_feedback['recommendations']:
            print(f"  • {rec}")

print(f"\n⏱️ Processing time: {response.total_time:.2f} seconds")

## Batch Query Processing

In [ ]:
# Process multiple queries to test system consistency
test_queries = [
    "What are the main challenges in implementing self-evaluating AI systems?",
    "How do RAG systems handle factual consistency and hallucination?",
    "Explain the role of tool validation in agentic AI workflows.",
    "What metrics should be used to evaluate prompt engineering effectiveness?"
]

batch_results = []

print("Processing batch queries...\n")

for i, query in enumerate(test_queries, 1):
    print(f"Query {i}: {query}")
    
    response = await ai_system.process_query(query)
    batch_results.append(response)
    
    # Show brief results
    print(f"Response: {response.response[:100]}...")
    
    if response.evaluation:
        score = response.evaluation.overall_score
        status = "✅" if response.evaluation.passed_thresholds else "⚠️"
        print(f"Score: {score:.2f} {status}")
    
    print(f"Time: {response.total_time:.2f}s\n")

# Summary statistics
avg_score = sum(r.evaluation.overall_score for r in batch_results if r.evaluation) / len(batch_results)
avg_time = sum(r.total_time for r in batch_results) / len(batch_results)
success_rate = sum(1 for r in batch_results if r.evaluation and r.evaluation.passed_thresholds) / len(batch_results)

print("="*50)
print("BATCH PROCESSING SUMMARY:")
print("="*50)
print(f"Queries processed: {len(batch_results)}")
print(f"Average score: {avg_score:.2f}")
print(f"Average processing time: {avg_time:.2f}s")
print(f"Success rate: {success_rate:.1%}")

## Session Statistics and Performance Analysis

In [ ]:
# Get comprehensive session statistics
session_stats = ai_system.get_session_stats()
response_history = ai_system.get_response_history()

if session_stats:
    print("📊 SESSION PERFORMANCE ANALYSIS:")
    print("="*40)
    print(f"Session ID: {session_stats['session_id']}")
    print(f"Start Time: {session_stats['start_time']}")
    print(f"Queries Processed: {session_stats['queries_processed']}")
    print(f"Successful Responses: {session_stats['successful_responses']}")
    print(f"Success Rate: {session_stats['successful_responses']/session_stats['queries_processed']*100:.1f}%")
    print(f"Average Score: {session_stats['average_score']:.2f}")
    print(f"Total Evaluation Time: {session_stats['total_evaluation_time']:.2f}s")

print("\n📈 DETAILED RESPONSE ANALYSIS:")
print("="*40)

for i, response_data in enumerate(response_history[-3:], 1):  # Show last 3 responses
    print(f"\nResponse {i}:")
    print(f"  Query: {response_data['query'][:60]}...")
    print(f"  Processing Time: {response_data['total_time']:.2f}s")
    
    if response_data.get('evaluation'):
        eval_data = response_data['evaluation']
        print(f"  Overall Score: {eval_data['overall_score']:.2f}")
        print(f"  Passed Thresholds: {'Yes' if eval_data['passed_thresholds'] else 'No'}")

## Configuration Customization

In [ ]:
# Create a custom configuration for specific use cases
from agentic_ai import SystemConfig, save_config, load_config
from pathlib import Path

# High-quality configuration (strict thresholds)
strict_config = SystemConfig()
strict_config.openai.api_key = os.environ["OPENAI_API_KEY"]
strict_config.openai.model = "gpt-4o-mini"
strict_config.openai.temperature = 0.1  # More deterministic

# Strict evaluation thresholds
strict_config.evaluation.faithfulness_threshold = 0.9
strict_config.evaluation.relevancy_threshold = 0.9
strict_config.evaluation.quality_threshold = 0.85

# Agent settings for careful processing
strict_config.agent.max_iterations = 8
strict_config.agent.max_revisions = 3
strict_config.agent.enable_self_revision = True

# Save configuration
config_path = Path("strict_config.yaml")
save_config(strict_config, config_path)

print(f"✅ Strict configuration saved to: {config_path}")

# Test with strict configuration
strict_ai = AdvancedAgenticAI(strict_config)
strict_ai.start_session("strict_mode_test")

test_query = "What are the most important considerations for deploying agentic AI in production environments?"
strict_response = await strict_ai.process_query(test_query)

print(f"\n🎯 STRICT MODE TEST:")
print(f"Query: {test_query}")
print(f"Response: {strict_response.response[:200]}...")

if strict_response.evaluation:
    print(f"\nStrict Evaluation Results:")
    print(f"  Overall: {strict_response.evaluation.overall_score:.3f}")
    print(f"  Faithfulness: {strict_response.evaluation.faithfulness_score:.3f} (threshold: 0.9)")
    print(f"  Relevancy: {strict_response.evaluation.relevancy_score:.3f} (threshold: 0.9)")
    print(f"  Passed: {'✅ YES' if strict_response.evaluation.passed_thresholds else '❌ NO'}")

## Export and Data Analysis

In [ ]:
# Export session data for analysis
from datetime import datetime

export_file = f"tutorial_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
ai_system.export_session(export_file)

print(f"📁 Session data exported to: {export_file}")

# Load and analyze the exported data
import json

with open(export_file, 'r') as f:
    session_data = json.load(f)

print(f"\n📊 EXPORTED DATA ANALYSIS:")
print(f"Export timestamp: {session_data['export_timestamp']}")
print(f"Total responses in history: {len(session_data['response_history'])}")
print(f"Knowledge base documents: {session_data['knowledge_base_stats']['total_documents']}")

# Analyze evaluation scores distribution
scores = [r['evaluation']['overall_score'] for r in session_data['response_history'] if r.get('evaluation')]

if scores:
    print(f"\n📈 Score Distribution:")
    print(f"  Min: {min(scores):.2f}")
    print(f"  Max: {max(scores):.2f}")
    print(f"  Mean: {sum(scores)/len(scores):.2f}")
    print(f"  High-quality responses (≥0.8): {sum(1 for s in scores if s >= 0.8)}/{len(scores)}")

## Conclusion

This tutorial demonstrated the key features of the Advanced Self-Evaluating Agentic AI System:

1. **Self-Evaluation**: Automatic quality assessment with multiple metrics
2. **Knowledge Management**: Dynamic knowledge base with search capabilities
3. **Performance Monitoring**: Comprehensive statistics and analysis
4. **Configuration Management**: Flexible settings for different use cases
5. **Data Export**: Session analysis and performance tracking

The system provides a robust foundation for building reliable agentic AI applications with built-in quality assurance and monitoring capabilities.

### Next Steps

- Experiment with different evaluation thresholds
- Add domain-specific knowledge to the knowledge base
- Integrate with external data sources
- Implement custom evaluation metrics
- Scale to production environments with monitoring